# Merging VA and DVD Datasets, and adding advice and transcript data

This notebook will walk through our process of cleaning the data sets and combining them.

In [ ]:
# loading necessary libararies
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from collections import defaultdict

In [ ]:
# load datasets
dvd_original = pd.read_csv("DVD_Original.csv") #include the version with the DVD MD info
va_original = pd.read_csv("VA_Original.csv")

# Viewing the two datasets

In [ ]:
dvd_original.head(3)

In [ ]:
va_original.head(3)

# Simple Variable Name Changes

The variables under this section are shared between the two datasets and only need a variable name change to match because the responses to these variables are coded the same. We have created dictionaries to map out these changes. 

In [ ]:
"""
    Maps variable names of one dataset to the other
    
    input: dataset
    output: dataset with new matching variable names
"""
def map_var_names(df, var_map):
    df_copy = df.copy()
    df_changed_names = df_copy.rename(index=str, columns=var_map)
    return df_changed_names
# NOTE: we make a copy of df to preserve the integrity of the original dataset

### Demographics

This section matches up the patient demographic variables between the two datasets.

In [ ]:
# maps DVD demo. var names to var names in VA
demo_map = {
    'demo_ID': 'ID', 'demo_age': 'age', 'demo_education': 'education', 'demo_race_arabme': 'arabme',
    'demo_race_hispanic': 'hispanic', 'demo_race_white': 'white', 'demo_race_black': 'black',
    'demo_race_native': 'native', 'demo_race_asian': 'asian', 'demo_race_pacific': 'pacific',
    'demo_race_other': 'raceother', 'demo_marry': 'marry'
}

### Physician Interaction

This section matches up the physician interaction variables between the two datasets. For example, the answer to the question, "My physician asked for my opinion about what type of treatment would be best for me" (Urosat10 / Opinion3).

In [ ]:
# maps DVD phys.interaction var names to var names in VA
phys_map = {
    'Urosat10': 'Opinion3', 'Urosat2_rev': 'Ask3', 'Urofeel3_t3': 'Info3',
    'Urofeel3_t3': 'Explain3', 'UroApt_partdec': 'Involve3', 'Urosat1': 'Satis3',
    'Dec3_best': 'Clear3', 'Urosat7': 'mdrespme', 'Urofeel6_t3': 'irespmd'
}

### Knowledge Questions

Patients were administered comprehension questions to ensure they had studied the background knowledge of prostate cancer, so they could make informed decisions when choosing treatment. This section matches up the patients' answers to these comprehension questions.

In [ ]:
# maps DVD knowledge var names to var names in VA
know_map = {
    'Know2_Dienot2': 'Dienot2', 'Know2_Wait2': 'Wait2', 'Know2_Sured2': 'Sured2',
    'Know2_Raded2': 'Raded2', 'Know2_Wwed2': 'Wwed2', 'Know2_Surpee2': 'Surpee2',
    'Know2_Radpee2': 'Radpee2', 'Know2_Wwpee2': 'Wwpee2'
}

### Anxiety Questions

Patients were asked questions concerning how anxious thinking about cancer made them. This section matches up the variable names for the answers to these questions.

In [ ]:
# maps DVD anxiety var names to var names in VA
anx_map = {
    'Anx11': 'Anx11', 'Anx12': 'Anx51', 'Anx13': 'Anx61',
    'Anx14': 'Anx71', 'Anx15': 'Anx91', 'Anx16': 'Anx111',
    'Anx21': 'Anx12', 'Anx22': 'Anx52', 'Anx23': 'Anx62',
    'Anx24': 'Anx72', 'Anx25': 'Anx92', 'Anx26': 'Anx112',
    'Anx31': 'Anx13', 'Anx32': 'Anx53', 'Anx33': 'Anx63',
    'Anx34': 'Anx73', 'Anx35': 'Anx93', 'Anx36': 'Anx113'
}

### MD Demographics

This section matches up demographic variables on doctors that are present in both datasets.

In [ ]:
# maps misc. DVD var names to var names in VA
md_demo_map = {
    'MD1_type': 'MD_type', 'MD1_age': 'MD_age',
    'MD1_gender': 'MD_gender', 'MD1_race': 'MD_race',
    'MD1_graduate': 'MD_yrgrad', 'MD1_spec': 'MD_specialty',
    'MD1_weeklyPT': 'MD_number_pts_wk', 'MD1_percentcare':'MD_percentpts'
}

### Miscellaneous

This section matches up miscellaneous variables that are present in both datasets.

In [ ]:
# maps misc. DVD var names to var names in VA
misc_map = {
    'DA2_DA_use_dichot': 'Timeda2', 'Know2_Whyww2': 'Whyww2',
    'Cancer_Gleason': 'gleason', 'Cancer_psa': 'psa1'
}

# Processed Variables

The variables under this section need more processing to match. We need to recode either columns in VA or DVD, or both, for the variable to correspond.

### Urologist Recommendation Binary
Variable representing whether physician provided a recommendation. In DVD, the variable 'Urorec_none' asks if the physician provided no recommendation, and therefore want to flip all responses (0->1, 1->0). In VA, the variable is coded as 1 (yes) and 2 (no), so we want all 2s to become 0. 

In [ ]:
def rec3_binary(df_d, df_v):
    df_dvd = df_d.copy()
    df_va = df_v.copy()
    # rename DVD column to match VA column name
    column_name = {'Urorec_none': 'Mdtxrec3'}
    df_dvd = df_dvd.rename(index=str, columns = column_name)
    # recode dvd column to go from "no physician recommendation" -> "yes physician recommendation" (flip responses)
    new_col1 = []
    for index, row in df_dvd.iterrows():
        if pd.isnull(row['Mdtxrec3']):
            new_col1.append(np.nan)
        elif row['Mdtxrec3'] == 1:
            new_col1.append(0) # 1 -> 0
        else:
            new_col1.append(1) # 0 -> 1
    df_dvd['Mdtxrec3'] = new_col1
    # recode va so that 2->0 (no) to match coding specified in VA codebook
    df_va.loc[(df_va['Mdtxrec3'] == 2), 'Mdtxrec3'] = 0
    return df_dvd, df_va

### DA
Maintaining original options from VA and DVD (nothing corresponds), and merging column to have 4 options. Now we have  control -> 2 and DVD -> 3 from DVD, and 0 -> NCCN and 1 -> MCC from VA. 

In [ ]:
def DA_type(df_d):
    df_dvd = df_d.copy()
    # rename DVD column to match VA column name
    column_name = {'DVD': 'DA1'}
    df_dvd = df_dvd.rename(index=str, columns = column_name)
    # we want anything in DVD that is 0 (control) to become 2 and 1 (DVD) to become 3 so that we have 4 distinct values
    df_dvd.loc[(df_dvd['DA1'] == 0), 'DA1'] = 2
    df_dvd.loc[(df_dvd['DA1'] == 1), 'DA1'] = 3
    return df_dvd

### Choice Made Binary
Have you and your doctor made a final decision about what type of treatment you will have for your prostate cancer? Recode VA coding to match DVD's (1->1, 2->0).

In [ ]:
def choice3_binary(df_d, df_v):
    df_dvd = df_d.copy()
    df_va = df_v.copy()
    # rename DVD column to match VA column name
    column_name = {'TxChoice3_yn': 'Finaltx3'}
    df_dvd = df_dvd.rename(index=str, columns = column_name)
    # we want anything in VA's Finaltx3 that is 2 (no) to become 0 (because 0 is no in DVD)
    df_va.loc[(df_va['Finaltx3'] == 2), 'Finaltx3'] = 0
    return df_dvd, df_va

### Discussing DA
Did you discuss the material in your decision aid with your physician? We are combining 2 (no) and -2 (refusal) to be recoded as 0 (no) in VA. We are adding refusal to "no" because it still means the patient did not discuss with the physician for whatever reason.

In [ ]:
def discussda_binary(df_d, df_v):
    df_dvd = df_d.copy()
    df_va = df_v.copy()
    # rename DVD column to match VA column name
    column_name = {'DA3_discussUro': 'Talkda3'}
    df_dvd = df_dvd.rename(index=str, columns = column_name)
    # we want everything coded as -2 (refusal) and 2 (no) in VA to be coded as no (0) to match DVD coding
    df_va.loc[(df_va['Talkda3'] == 2), 'Talkda3'] = 0
    df_va.loc[(df_va['Talkda3'] == -2), 'Talkda3'] = 0
    return df_dvd, df_va

### Treatment Decision at time 3 vs Treatment Received
Create a new variable, decision3_vs_received, that sees whether treatment choice specified at time 3 matched the treatment received. In VA, this variable already exists under the name TxgotTx3cc, and in DVD this variable needs to be created by comparing TxChoice3_decided with TxGot_pt_orig to see if they match. 

In [ ]:
def decision3_vs_received(df_d):
    df_dvd = df_d.copy()
    matched = []
    for i in range(len(df_dvd)): # check if TxChoice3_decided matches TxGot_pt_orig in DVD
        if (pd.isnull(df_dvd.TxChoice3_decided[i]) == False) and (df_dvd.TxChoice3_decided[i] == df_dvd.TxGot_pt_orig[i]):
            matched.append(1)
        else:
            matched.append(0)
    df_dvd['TxgotTx3cc'] = matched # create new variable that matches the VA variable name
    return df_dvd

### Treatment Received 
This variable represents the final treatment received. We recode VA's txgot variable to match the ordering in DVD (1: Active surveillance, 2: surgery, 3: e.b. Radiation, 4: brachytherapy, 5: other). Furthermore, adjuvant hormone therapy in VA becomes the other option (since that is not in DVD, and DVD has an other option).

In [368]:
# VA txgot - 1: surgery, 2: e.b. Radiation, 3: brachytherapy, 4: adjuvant hormone therapy, 5: active surveillance
# DVD TxGot_pt_orig - 1: Active surveillance, 2: surgery, 3: e.b. Radiation, 4: brachytherapy, 5: other
def treatment_received(df_d, df_v):
    df_dvd = df_d.copy()
    df_va = df_v.copy()
    # rename DVD column name to match VA column name
    column_name = {'TxGot_pt_orig': 'txgot'}
    df_dvd = df_dvd.rename(index=str, columns = column_name)
    # we want options 1->2, 2->3, 3->4, 5->1, 4->5 in VA to match DVD coding
    new_col = []
    for index, row in df_va.iterrows():
        if pd.isnull(row['txgot']):
            new_col.append(np.nan)
        elif row['txgot'] == 1: # 1->2
            new_col.append(2)
        elif row['txgot'] == 2: # 2->3
            new_col.append(3)
        elif row['txgot'] == 3: # 3->4
            new_col.append(4)
        elif row['txgot'] == 5: # 5->1
            new_col.append(1)
        else: # adjuvant hormone therapy will be considered other to correspond with DVD coding (4->5)
            new_col.append(5)
    df_va['txgot'] = new_col
    return df_dvd, df_va

### Treatment Received Binary
Convert treatment received into a binary variable which is 1 if the treatment is active surveillance, and 0 otherwise. 

In [367]:
def treatment_received_binary(df_merged):
    df_merged_copy = df_merged.copy()
    bin_vars = []
    for index, row in df_merged_copy.iterrows():
        if pd.isnull(row['txgot']):
            bin_var = np.nan
        elif row['txgot'] == 1:
            bin_var = 1
        else:
            bin_var = 0
        bin_vars.append(bin_var)
    df_merged_copy['txgot_binary'] = bin_vars
    return df_merged_copy

### Treatment Lean
This variable represents the variable they are leaning towards at time 3. Both variables have to be recoding. In DVD, both option 5 (waiting to make decision) and option 6 (other) combine to become option 5 (which represents other).  In VA, option 4 (adjuvant hormone therapy), option 6 (experimental therapies), and option 7 (other) combine to become option 5 (which represents other) and options -1 and -2 (which are not in the codebook) are considering missing data (np.nan). The new representation will be 1: Active surveillance, 2: surgery, 3: e.b. Radiation, 4: brachytherapy, 5: other.

In [366]:
# VA - 1: surgery, 2: e.b. Radiation, 3: brachytherapy, 4: adjuvant hormone therapy, 5: active surveillance, 6: experimental therapies, 7: other
# DVD - 1: active surveillance, 2: surgery, 3: e.b. radiation, 4: brachytherapy, 5: I am waiting to make my decision, 6: other
def treatment_lean(df_d, df_v):
    df_dvd = df_d.copy()
    df_va = df_v.copy()
    # rename DVD column name to match VA column name
    column_name = {'TxChoice3_lean': 'Txlean3'}
    df_dvd = df_dvd.rename(index=str, columns = column_name)
    # recode DVD so that options 5,6 become other (essentially, 6 -> 5)
    df_dvd.loc[(df_dvd['Txlean3'] == 6),'Txlean3'] = 5
    # we want options 1->2, 2->3, 3->4, 5->1, 4,6,7 become other (essentially 4,6,7->5) in VA to match DVD
    new_col = []
    for index, row in df_va.iterrows():
        if pd.isnull(row['Txlean3']) or row['Txlean3'] == -1 or row['Txlean3'] == -2:
            new_col.append(np.nan) # -1 and -2 -> np.nan
        elif row['Txlean3'] == 1: # 1->2
            new_col.append(2)
        elif row['Txlean3'] == 2: # 2->3
            new_col.append(3)
        elif row['Txlean3'] == 3: # 3->4
            new_col.append(4)
        elif row['Txlean3'] == 5: # 5->1
            new_col.append(1)
        else: # 4,6,7 -> 5
            new_col.append(5) 
    df_va['Txlean3'] = new_col
    return df_dvd, df_va

# Recoding Advice

First we will recode in VA. In VA, the advice columns are: 'ActiveSurveillance', 'Surgery', and 'Radiation'. These variables are coded from -2 to 2, where 2 is a strongly advised and -2 is strongly advised against. We are combining these into one column that represents the most recommended option(s), where we include ties. For example, if R and S both receive a recommendation of +1 and A receives a recommendation of -1, then we would include both R and S.

In [369]:
def recode_advice_va(df):
    advice_col = []
    doc_col = []
    for index, row in df.iterrows():
        advice = ''
        doc = np.nan
        max_val = max([row['ActiveSurveillance'], row['Surgery'], row['Radiation']])
        for treatment in ['ActiveSurveillance', 'Surgery', 'Radiation']:
            if row[treatment] == max_val and pd.isnull(row[treatment]) == False:
                advice += treatment[0]
                doc = 'U'
        if advice == '':
            advice = np.nan 
        advice_col.append(advice)
        doc_col.append(doc)
    df['Advice1']  = advice_col
    df['Doc1'] = doc_col
    return df

Now, we will recode in DVD. In DVD, there is a separate advice spreadsheet, and the main variables we are interested in are: 'DVD_ActiveSurveillance', 'DVD_Radiation', and 'DVD_Surgery'. These are are on a scale from 1 (most advised option) to 3 (least advised option). If there are tie breakers, we include all tied options. For example, if two options tied for first place, this would result in the values 1.5 (A), 1.5 (R), and 3 (S) (note: the 3 advice variables should add up to 6 if it's not empty) and we would include both A and R.  

In [370]:
def recode_advice_dvd(df):
    advice = defaultdict(lambda: {'Advice1': np.nan, 'Advice2': np.nan, 'Doc1': np.nan, 'Doc2': np.nan, 'as1': np.nan, 'rad1': np.nan, 'sur1': np.nan, 'as2': np.nan, 'rad2': np.nan, 'sur2': np.nan})
    for index, row in df.iterrows():
        id_ = row['ID']
        p_id = id_[1:5]
        as_ = row['DVD_ActiveSurveillance']
        rad_ = row['DVD_Radiation']
        sur_ = row['DVD_Surgery']
        # compute the advice (most recommended option)
        max_val = max([as_, rad_, sur_])
        adv = ''
        for treatment in ['DVD_ActiveSurveillance', 'DVD_Surgery', 'DVD_Radiation']:
            if row[treatment] == max_val and pd.isnull(row[treatment]) == False:
                adv += treatment[4]
        if adv == '':
            adv = np.nan
        if id_[8] == '1':
            advice[p_id]['Doc1'] = id_[6]
            advice[p_id]['as1'] = as_
            advice[p_id]['rad1'] = rad_
            advice[p_id]['sur1'] = sur_
            advice[p_id]['Advice1'] = adv
        else:
            advice[p_id]['Doc2'] = id_[6]
            advice[p_id]['as2'] = as_
            advice[p_id]['rad2'] = rad_
            advice[p_id]['sur2'] = sur_
            advice[p_id]['Advice2'] = adv
    return advice

# Actual Merging (credit to Grant's code)

We will finally merge the two datasets according to the following steps:
1. Run each dataset through their processing functions written above
2. Find shared variables
3. For the variables unique to each dataset, mark with a prefix (VA_ or DVD_)
4. Concatenate the two datasets

First, we have to find the variable names shared by the two datasets prior to processing because these variable names may interfere when we find the shared variables after processing.

The pre-processed shared variables consist of anxiety question variables for each of the datasets. These variables don't necessarily correspond to the same question between the datasets. However, looking at the mapping for the anxiety question section above, we see that most of the pre-processed shared variables have been mapped to the correct corresponding question. The variables that remain are: <br />

Anx41, Anx42, Anx43, Unnamed: 0, filter_$ <br />

When constructing the pool of shared variables, these variables will be taken out and given prefixes denoting their respective studies.

In [ ]:
"""
    Takes out the "faux" shared variables
    
    input: set of shared variables
    output: set of shared variables without variables above
"""
def filter_set(shared_set):
    shared_set_copy = shared_set.copy()
    
    fake_vars = ['Anx41', 'Anx42', 'Anx43', 'Unnamed: 0', 'filter_$']
    for var in fake_vars:
        shared_set_copy.discard(var)
    return shared_set_copy

While we're at it, let's make a function that adds prefixes to the unique variables of the datasets.

In [ ]:
# helper function for prefix adder below
def prefix_helper(x, prefix, shared_vars):
    if x in shared_vars:
        return x
    else: 
        return prefix + x

In [ ]:
"""
    Adds prefixes to the unique variables in each dataset
    
    input: dataframe, dataset ("va" or "dvd"), set of shared vars
    output: dataframe with the unique variables prefixed
"""
def add_prefix(df, dataset, shared_vars):
    df_copy = df.copy()
    
    prefix = ''
    if dataset == 'va':
        prefix = 'VA_'
    if dataset == 'dvd':
        prefix = 'DVD_'
        
    df_copy.columns = list(map(lambda x: prefix_helper(x, prefix, shared_vars),
                              df.columns))
    
    return df_copy

The next function will run each dataset through their processing functions as defined earlier.

In [ ]:
"""
    Processes datasets to get ready for merging
    
    input: both original dataframes
    output: datasets ready to be merged
"""

def process_data(df1, df2):
    va = df1.copy()
    dvd = df2.copy()
    doctor_advice = pd.read_csv('DVD_Advice.csv')
    
    # constructing  advice variable
    advice_dict = recode_advice_dvd(doctor_advice)
    advice_d = pd.DataFrame.from_dict(advice_dict, orient='index')
    advice_d.index.name = 'demo_ID'
    advice_d.reset_index(level=0, inplace=True)
    dvd_new = dvd.merge(advice_d, how = 'left', on='demo_ID') # have to join advice dataframe with dvd dataframe
    va_new = recode_advice_va(va)
    
    # dvd simple name changes
    demo_dvd = map_var_names(dvd_new, demo_map)
    phys_dvd = map_var_names(demo_dvd, phys_map)
    know_dvd = map_var_names(phys_dvd, know_map)
    anx_dvd = map_var_names(know_dvd, anx_map)
    md_dvd = map_var_names(anx_dvd, md_demo_map)
    misc_dvd = map_var_names(md_dvd, misc_map)
    
    # VARIABLES THAT REQUIRE PROCESSING
    
    # Urologist Recommendation
    urorec_dvd, urorec_va = rec3_binary(misc_dvd, va_new)
    
    # DA type
    da_dvd = DA_type(urorec_dvd)
    
    # Choice 3
    choice3_dvd, choice3_va = choice3_binary(da_dvd, urorec_va)
    
    # Discussing DA
    dda_dvd, dda_va = discussda_binary(choice3_dvd, choice3_va)
    
    # Decision3 vs. Received
    tx3txgot_dvd = decision3_vs_received(dda_dvd)
    
    # Treatment Final
    txgot_dvd, txgot_va = treatment_received(tx3txgot_dvd, dda_va)
    
    # Treatment Lean
    final_dvd, final_va = treatment_lean(txgot_dvd, txgot_va)
    
    return final_va, final_dvd

Apologies if this processing function seems convoluted because of the chain of function calls. However, I think having each step of processing in its own function will make it easier to add new steps of processing down the line.

Next, we write a function that finds the shared variables and makes the distinction between shared and unique variables.

In [ ]:
"""
    Finds set of shared variables and marks shared and distinct variables 
    in each dataframe
    
    input: processed va, processed dvd (result of process_data)
    output: two ready to merge dataframes
"""
def process_shared(df1, df2):
    va = df1.copy()
    dvd = df2.copy()
    
    # finds the shared variables between the datasets
    shared_vars = set(va) & set(dvd)
    filtered_vars = filter_set(shared_vars)
    
    # marks dataframes with prefixes for unique variables
    final_va = add_prefix(va, 'va', filtered_vars)
    final_dvd = add_prefix(dvd, 'dvd', filtered_vars)
    
    return final_va, final_dvd, filtered_vars

Finally, we will merge the two processed datasets into a beautiful, singular dataset.

In [ ]:
"""
    Takes two original datasets and returns merged dataset
    
    input: original va, original dvd
    output: merged dataframe
"""
def merge(va_original, dvd_original):
    # runs original dataframes through above processing functions
    processed_va, processed_dvd = process_data(va_original, dvd_original)
    tagged_va, tagged_dvd, filtered_vars = process_shared(processed_va, processed_dvd)
    
    merged_df = pd.concat([tagged_va, tagged_dvd])
    merged_df_as = treatment_received_binary(merged_df) # convert txgot into a binary variable for classification (1 for AS, 0 otherwise)
    return merged_df_as

# Moment of Truth

In [ ]:
df_merged = merge(va_original, dvd_original)

# Merging Transcript Data (LAST STEP!!!)

In [ ]:
df_transcripts = pd.read_csv('all_transcripts.csv')
df_transcripts = df_transcripts.drop(columns=['Unnamed: 0'])
df_transcripts.rename(columns = {'patient_id':'ID'}, inplace=True)

In [ ]:
transcripts_merged = df_merged.merge(df_transcripts, how='left', on='ID')

In [ ]:
transcripts_merged.to_csv('all_data_merged.csv')

### Create a dataframe with shared vars only

In [ ]:
# create a dataframe that only include shared variables, md info, advice, and transcript data (nothing unique to VA or DVD)
transcripts_merged_sub = transcripts_merged[['Advice1', 'Anx11', 'Anx111', 'Anx112', 'Anx113', 'Anx12', 'Anx13', 'Anx51', 'Anx52','Anx53','Anx61','Anx62','Anx63','Anx71','Anx72','Anx73','Anx91','Anx92','Anx93','Ask3','Clear3','DA1','Dienot2','Doc1','Explain3','Finaltx3','ID','Involve3','MD_age','MD_gender','MD_number_pts_wk','MD_percentpts','MD_race','MD_specialty','MD_type','MD_yrgrad','Mdtxrec3','Opinion3','Raded2','Radpee2','Satis3','Sured2','Surpee2','Talkda3','Timeda2','TxgotTx3cc','Txlean3','Wait2','Whyww2','Wwed2','Wwpee2','age','arabme','asian','black','education','gleason','hispanic','irespmd','marry','mdrespme','native','pacific','psa1','raceother','txgot','txgot_binary','white', 'Convo_1', 'Convo_2', 'Dataset', 'Doctor_1', 'Doctor_2']]

In [ ]:
transcripts_merged_sub.to_csv('shared_data_merged.csv')